In [50]:
print("Starting imports...")
import numpy as np
import matplotlib.pyplot as plt
from numba import cuda, float64, complex128
from numba.cuda import jit as cuda_jit
import math

print("Importing few...")
import few

from few.trajectory.inspiral import EMRIInspiral
from few.trajectory.ode import KerrEccEqFlux
from few.amplitude.ampinterp2d import AmpInterpKerrEccEq
from few.summation.interpolatedmodesum import InterpolatedModeSum 


from few.utils.ylm import GetYlms

from few import get_file_manager

from few.waveform import FastKerrEccentricEquatorialFlux

from few.utils.geodesic import get_fundamental_frequencies

from few.utils.constants import YRSID_SI

import os
import sys

# Change to the desired directory
os.chdir('/nfs/home/svu/e1498138/localgit/FEWNEW/work/')

# Add it to Python path
sys.path.insert(0, '/nfs/home/svu/e1498138/localgit/FEWNEW/work/')

print("Importing GWfuncs...")
import GWfuncs
# import gc
# import pickle
print("Importing cupy...")
import cupy as cp

print("Configuring few...")
# tune few configuration
cfg_set = few.get_config_setter(reset=True)
cfg_set.set_log_level("info")

print("Importing dynesty...")
import dynesty

Starting imports...
Importing few...
Importing GWfuncs...
Importing cupy...
Configuring few...
Importing dynesty...


In [51]:
for backend in ["cpu", "cuda11x", "cuda12x", "cuda", "gpu"]: 
    print(f" - Backend '{backend}': {"available" if few.has_backend(backend) else "unavailable"}")  

 - Backend 'cpu': available
 - Backend 'cuda11x': unavailable
 - Backend 'cuda12x': available
 - Backend 'cuda': available
 - Backend 'gpu': available


In [52]:
# Initialize parameters and backend
N_traj = 500
use_gpu = True 
T = 1 # year
T_sec = T * YRSID_SI
dt = 10
delta_T = T_sec / N_traj # define coarser sampling time step

In [53]:
# Waveform generation attributes
# NOTE: honestly the only diff is in the EMRIInspiral class, which has a different npoints flag
# otherwise only need to initialize waveform_gen with FastKerrEccentricEquatorialFlux
# TODO: ? 
traj = EMRIInspiral(func=KerrEccEqFlux, force_backend="cuda12x", use_gpu=use_gpu, npoints = N_traj) #theres npoints flag here
amp = AmpInterpKerrEccEq(force_backend="cuda12x") # default lmax=10, nmax=55
interpolate_mode_sum = InterpolatedModeSum(force_backend="cuda12x")
ylm_gen = GetYlms(include_minus_m=False, force_backend="cuda12x")

In [54]:
#Generating data (true)

m1_o = 1e6
m2_o = 1e1
a_o = 0.3
p0_o = 12
e0_o = 0.1
xI_o = 1.0
theta_o = np.pi/3  # polar viewing angle
phi_o = np.pi/4  # azimuthal viewing angle
dist = 1 # Gpc

In [55]:
(t, p, e, x, Phi_phi, Phi_theta, Phi_r) = traj(m1_o, m2_o, a_o, p0_o, e0_o, xI_o, T=T, dt=delta_T, upsample=True) 


In [56]:
# Generate mode frequencies
OmegaPhi, OmegaTheta, OmegaR = get_fundamental_frequencies(a_o, p, e, x) #could use GPU here but im running to mismatch probs


In [57]:
# Get mode labels
mode_labels = [f"({l},{m},{n})" for l,m,n in zip(amp.l_arr, amp.m_arr, amp.n_arr)]

In [58]:
l_cpu = amp.l_arr.get()
m_cpu = amp.m_arr.get()
n_cpu = amp.n_arr.get()

In [59]:
gw_frequencies_per_mode = []

for idx in range(len(mode_labels)):
    # TODO: do convert this so everything is in either CPU/GPU?
    # l = amp.l_arr[idx]
    # m = amp.m_arr[idx] 
    # n = amp.n_arr[idx]
    
    l = l_cpu[idx]
    m = m_cpu[idx] 
    n = n_cpu[idx]
    
    # Calculate GW frequencies
    # k = 0 for equatorial case
    f_gw = m * OmegaPhi + n * OmegaR
    
    gw_frequencies_per_mode.append(f_gw)# Generate mode frequencies

In [60]:
gw_phase_per_mode = []
for idx in range(len(mode_labels)):
    # l = amp.l_arr[idx]
    # m = amp.m_arr[idx] 
    # n = amp.n_arr[idx]

    l = l_cpu[idx]
    m = m_cpu[idx] 
    n = n_cpu[idx]
    
    # Calculate GW phases per mode
    phi_mode = m * Phi_phi + n * Phi_r
    
    gw_phase_per_mode.append(phi_mode)

In [61]:
gwf = GWfuncs.GravWaveAnalysis(T, dt)
factor = gwf.dist_factor(dist, m2_o)

In [62]:
# Get amplitudes along trajectory
teuk_modes = amp(a_o, p, e, x)

# Get Ylms
ylms = ylm_gen(amp.unique_l, amp.unique_m, theta_o, phi_o).copy()[amp.inverse_lm]

In [63]:
mode_selector = GWfuncs.ModeSelector(teuk_modes=teuk_modes, amp=amp, gw_frequencies=gw_frequencies_per_mode, gw_phases= gw_phase_per_mode, delta_T=delta_T, factor=factor, gwf=gwf)

In [66]:
m0mask = amp.m_arr_no_mask != 0

total_power = gwf.calc_power(teuk_modes, ylms, m0mask)
total_power_cpu = total_power.get()
M_mode = 100
mp = list(zip(mode_labels, total_power_cpu)) 
# Sort by power and pick top M modes
mp_sorted = sorted(mp, key=lambda x: x[1], reverse=True)[:M_mode]
mp_modes = [x[0] for x in mp_sorted]
mp_power = [x[1] for x in mp_sorted]

In [68]:
selected_modes, selected_indices, selected_labels, counts, final_ips = mode_selector.select_modes_power(
    power_values=mp_power, mode_labels=mp_modes, original_mode_labels=mode_labels, M_sel=50, inner_threshold=0.01
)

Step 0: Selected strongest mode h_0: (2,2,0) with power value 1.3592e+01
Using power inner product calculation.
Inner product: 1280.725840

--- Iteration 1 ---
Currently have 1 selected modes, target is 50
Candidate mode h_j': (2,2,1) with power value 9.3625e-01
  Calculating inner product with selected mode [1165] and hj_prime_idx: 1166
0.0020743028637069867
  calc_inner with selected mode 0 ((2,2,0)): 0.002074
  ACCEPTED: Max inner product 0.002074 < 0.01
  Added mode: (2,2,1) (Total selected: 2)

--- Iteration 2 ---
Currently have 2 selected modes, target is 50
Candidate mode h_j': (3,3,0) with power value 8.5560e-01
  Calculating inner product with selected mode [1165] and hj_prime_idx: 1498
0.0003130074030478932
  calc_inner with selected mode 0 ((2,2,0)): 0.000313
  Calculating inner product with selected mode [1166] and hj_prime_idx: 1498
0.0002403232465312825
  calc_inner with selected mode 1 ((2,2,1)): 0.000240
  ACCEPTED: Max inner product 0.000313 < 0.01
  Added mode: (3,3,0